In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot = True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [25]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128
X = tf.placeholder(tf.float32, [None, n_input]) # 실제 디이터 입력
Z = tf.placeholder(tf.float32, [None, n_noise]) # 노이즈 입력

In [26]:
# n:noise : 128  / n_hidden : 256 => 128x256
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))

# 256 x 784
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev = 0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden,1], stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [31]:
# 생성기
# cost function : 확률적 함수
# kullback-Leibler( KL-Divergence) : GAN, VAE  :: 분포의 차이를 확인하는 함수
#  * MSE , Entropy : 분류
# 생성기 : noiser가 input데이터, cost function : KL-Divergence

def generator(noise_z) :  # 노이즈가 128x128로 들어옴
    hidden = tf.nn.relu( tf.matmul(noise_z , G_W1 ) + G_b1) # 128 x 256
    output = tf.nn.sigmoid( tf.matmul(hidden, G_W2) + G_b2) # 128 x 784
    return output # 784 사이즈의 이미지 생성


#판별기
def discriminator(inputs) 00000000000000000000000000000000000000
:   # 실제 이미지의 분포를 확인
    hidden = tf.nn.relu( tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid( tf.matmul(hidden, D_W2) + D_b2)
    return output


def get_noise(batch_size, n_noise):
    return np.random.normal(size = (batch_size , n_noise))

In [28]:
G = generator(Z)  # 노이즈로 이미지 생성
D_gene = discriminator(G) # 노이즈로 생성돤 이미지를 판별( 분포를 확인 )
D_real = discriminator(X) # 실제 이미지의 분포를 확인

# log 사용 이유 : 확률값에 log를 취함으로서 정보량을 구함
# => 확률이 높아지면 정보량이 작아짐
# => 확률이 낮아지면 정보량이 커짐     ==> tfidf의 논리와 유사 
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1- D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene)) # 최대우도 추정법으로 가장 적합한 분포가 될 수 있도록 함
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]
# 최대화하기 위해
# 역전파 변수를 지정
# 판별기의 loss와 생성기의 loss를 minimize
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list) 
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

%matplotlib inline
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/ batch_size)
loss_val_D , loss_val_G = 0 ,0

In [29]:
#

for epoch in range(total_epoch) :
    for i in range(total_batch) :
        batch_xs , batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise) # 128x128
        # 실제 이미지 분포를 확인하기 위함
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict = {X: batch_xs, Z: noise})
        # 아마자룰 생성( noise : 128x128)
        _, loss_val_G = sess.run([train_G, loss_G],
                                feed_dict={Z: noise})
    print("Epoch : ", '%04d' % epoch,
         'D loss : {:.4}'.format(loss_val_D),
         "G loss : {:.4}".format(loss_val_G))
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        # 노이즈로부터 이미지를 생성 : generator
        noise = get_noise(sample_size , n_noise)   #10 x 128 개의 노이즈
        samples = sess.run(G, feed_dict={Z: noise}) 
        fig, ax =  plt.subplots(1, sample_size , figsize=(sample_size,1))
        for i in range(sample_size):
            ax[i].set_axis_off()     # 128 => 784 이미지 사이즈로 생성
            ax[i].imshow(np.reshape(samples[i], (28,28)))
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),
                   bbox_inches = 'tight')
        plt.close(fig)
print("최적화 완료")

Epoch :  0000 D loss : -0.7055 G loss : -1.866
Epoch :  0001 D loss : -0.1817 G loss : -2.844
Epoch :  0002 D loss : -0.2073 G loss : -2.84
Epoch :  0003 D loss : -0.5438 G loss : -1.68
Epoch :  0004 D loss : -0.4954 G loss : -1.51
Epoch :  0005 D loss : -0.2321 G loss : -2.475
Epoch :  0006 D loss : -0.1313 G loss : -3.084
Epoch :  0007 D loss : -0.187 G loss : -3.162
Epoch :  0008 D loss : -0.2324 G loss : -2.523
Epoch :  0009 D loss : -0.3179 G loss : -2.584
Epoch :  0010 D loss : -0.3462 G loss : -2.635
Epoch :  0011 D loss : -0.459 G loss : -1.96
Epoch :  0012 D loss : -0.5327 G loss : -2.013
Epoch :  0013 D loss : -0.401 G loss : -2.281
Epoch :  0014 D loss : -0.3553 G loss : -2.415
Epoch :  0015 D loss : -0.3086 G loss : -2.583
Epoch :  0016 D loss : -0.3976 G loss : -2.575
Epoch :  0017 D loss : -0.4277 G loss : -2.805
Epoch :  0018 D loss : -0.2863 G loss : -2.861
Epoch :  0019 D loss : -0.3389 G loss : -2.896
Epoch :  0020 D loss : -0.3075 G loss : -2.973
Epoch :  0021 D loss